In [1]:
#https://www.gairuo.com/m/pandas-trend-continuous-change
import pandas as pd
import numpy as np

In [2]:
np.random.seed(4096)
df = pd.DataFrame(np.random.randint(0, 100, (8, 6)),
                  columns=pd.array([*'123456'])+'月'
                 )
df

,1月,2月,3月,4月,5月,6月
0,36,40,89,40,90,88
1,23,78,70,62,30,17
2,23,42,68,49,87,96
3,66,75,45,96,61,20
4,22,71,74,65,75,95
5,2,41,64,72,89,96
6,35,93,45,42,75,18
7,99,83,1,50,85,64


In [3]:
df.iloc[4, 1:] = df.iloc[4,1]
df

,1月,2月,3月,4月,5月,6月
0,36,40,89,40,90,88
1,23,78,70,62,30,17
2,23,42,68,49,87,96
3,66,75,45,96,61,20
4,22,71,71,71,71,71
5,2,41,64,72,89,96
6,35,93,45,42,75,18
7,99,83,1,50,85,64


In [4]:
temp_ser = df.loc[1]
temp_ser

1月    23
2月    78
3月    70
4月    62
5月    30
6月    17
Name: 1, dtype: int32

In [5]:
temp_ser.diff()

1月     NaN
2月    55.0
3月    -8.0
4月    -8.0
5月   -32.0
6月   -13.0
Name: 1, dtype: float64

In [6]:
temp_ser = temp_ser.diff().apply(np.sign)
temp_ser

1月    NaN
2月    1.0
3月   -1.0
4月   -1.0
5月   -1.0
6月   -1.0
Name: 1, dtype: float64

In [7]:
type_str = {1: '增长', -1: '降低', 0: '持平'}.get(temp_ser[-1])
type_str

'降低'

In [8]:
temp_ser = (temp_ser == temp_ser[-1])[::-1]
temp_ser

6月     True
5月     True
4月     True
3月     True
2月    False
1月    False
Name: 1, dtype: bool

In [9]:
temp_ser.cumprod()   #累积乘积

6月    1
5月    1
4月    1
3月    1
2月    0
1月    0
Name: 1, dtype: int32

In [10]:
num = temp_ser.cumprod().sum()
num

4

In [11]:
f'连续{num}个月{type_str}'

'连续4个月降低'

In [12]:
def func(ser: pd.Series):
    ser = ser.diff().apply(np.sign)
    type_str = {1: '增长', -1: '降低', 0: '持平'}.get(ser[-1])
    ser = (ser == ser[-1])[::-1]
    num = ser.cumprod().sum()
    return f'连续{num}个月{type_str}'

In [13]:
df.assign(连续增减=df.apply(func, axis=1))

,1月,2月,3月,4月,5月,6月,连续增减
0,36,40,89,40,90,88,连续1个月降低
1,23,78,70,62,30,17,连续4个月降低
2,23,42,68,49,87,96,连续2个月增长
3,66,75,45,96,61,20,连续2个月降低
4,22,71,71,71,71,71,连续4个月持平
5,2,41,64,72,89,96,连续5个月增长
6,35,93,45,42,75,18,连续1个月降低
7,99,83,1,50,85,64,连续1个月降低


In [20]:
#迭代方法
def func(ser: pd.Series):
    ser = iter(ser[::-1])
    cur, nxt = next(ser), next(ser)
    type_str = {1: '增长', -1: '降低', 0: '持平'}.get(cur)
    n = 1
    while cur == nxt:
        n += 1
        cur = nxt
        nxt = next(ser)
    return f'连续{n}个月{type_str}'
lst = (         
    df.diff(axis=1)
    .applymap(np.sign)
    .apply(func, axis=1)
)
df.assign(连续增减=lst)

,1月,2月,3月,4月,5月,6月,连续增减
0,36,40,89,40,90,88,连续1个月降低
1,23,78,70,62,30,17,连续4个月降低
2,23,42,68,49,87,96,连续2个月增长
3,66,75,45,96,61,20,连续2个月降低
4,22,71,71,71,71,71,连续4个月持平
5,2,41,64,72,89,96,连续5个月增长
6,35,93,45,42,75,18,连续1个月降低
7,99,83,1,50,85,64,连续1个月降低


In [24]:
#累加的思路
(         
    df.diff(axis=1)
    .applymap(np.sign)
    .set_index('6月', drop=False)
    .apply(lambda x: x!=x[-1], axis=1)
    .apply(lambda x: x[::-1].cumsum(), axis=1)
    .apply(lambda x: sum(x==0), axis=1)
    .rename({1: '增长', -1: '降低', 0: '持平'})
)


6月
降低    1
降低    4
增长    2
降低    2
持平    4
增长    5
降低    1
降低    1
dtype: int64

In [25]:
df.assign(连续增减='连续'+_map.astype(str).values+'个月'+_map.index)

NameError: name '_map' is not defined